# Generating centroids within Fremont neighborhoods

### Goal of the notebook

We'll slice neighborhood polygons into smaller parts by using `geovoronoi` module - this creates Voronoi diagrams within boundaries of each of the neighborhood boundaries.

***

**Outputs:**

Internal Centroid Zones:
- neighborhoods_centroid_zones.csv

**Inputs:**

Fremont neighborhoods:
- Neighborhoods.shp

***

In [1]:
# --- Global variables

# Setting up the Coordinate Reference Systems up front in the necessary format.
epsg = 4326
crs_degree = {'init': 'epsg:4326'} # CGS_WGS_1984 (what the GPS uses)

# --- Paths

# Root path of Fremont Dropbox
import os
import sys
# We let this notebook to know where to look for fremontdropbox module
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from fremontdropbox import get_dropbox_location
# Root path of the Dropbox business account
dbx = get_dropbox_location()

# Temporary! Location of the folder where the restructuring is currently happening
data_path = dbx + '/Private Structured data collection'
aux_files = data_path+'/Data processing/Auxiliary files'

# Processing output path
output_path = aux_files + '/OD demand'

## 1. Loading files and checking if the projection is correct (EPSG 4326)

In [2]:
# https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/
import sys
!conda install --yes --prefix {sys.prefix} pandas geopandas numpy

Solving environment: done

# All requested packages already installed.



In [3]:
import sys
!{sys.executable} -m pip install keplergl geovoronoi

In [4]:
from geopandas import GeoDataFrame

neighborhoods_shp_old_projection = GeoDataFrame.from_file(data_path + '/Data processing/Raw/Demand/OD demand/Fremont neighborhoods/Neighborhoods.shp')

neighborhoods_shp = neighborhoods_shp_old_projection.to_crs('epsg:4326')

assert(neighborhoods_shp.crs == 'epsg:4326'),'Neighborhoods shapefile Coordinate Reference System missmatch! CRS is not EPSG:4326.'
print('Neighborhoods shapefile Coordinate Reference System is epsg:4326, which is good!')

neighborhoods_shp.head()

Neighborhoods shapefile Coordinate Reference System is epsg:4326, which is good!


,OBJECTID,REGION,NAME,HISTORIC,UPD_DIV,UPD_PERSON,UPD_DATE,NSA_KEY,ACRES,geometry
0,1,01,Lakes and Birds,None,FREMONT GIS,RCHAN,20040617,1,317.471,"POLYGON ((-122.05039 37.58380, -122.05312 37.5..."
1,2,02,Ardenwood,None,FREMONT GIS,RCHAN,20040617,2,1570.961,"POLYGON ((-122.06751 37.54066, -122.06782 37.5..."
2,3,03,Northgate,None,FREMONT GIS,RCHAN,20040617,3,860.293,"POLYGON ((-122.05048 37.58394, -122.05026 37.5..."
3,4,04,Cabrillo,None,FREMONT GIS,RCHAN,20040617,4,746.339,"POLYGON ((-122.02642 37.55096, -122.02762 37.5..."
4,5,05,Brookvale,None,FREMONT GIS,RCHAN,20040617,5,1173.773,"POLYGON ((-122.00703 37.58295, -122.00699 37.5..."


***

## 2. Split neighborhoods into parts

We're going to use `geovoronoi` module here to do the splitting. We generate random points within each neighborhood boundaries and they will used as input centroids, from which Voronoi diagrams will be generated.

These Voronoi diagrams will then be converted into `Polygon`s. We'll generate new `GeoDataFrame` which will contain all of the new generated `Polygons` and set `CentroidID` and `NeighborhoodID` to each one of them.

In [5]:
import numpy as np
import geopandas as gpd
from demandutils import voronoi_within_neighborhood

np.random.seed(123)

partitioned_neighborhood_centroids = gpd.GeoDataFrame()

for neighborhood_index, neighborhood_name in enumerate(neighborhoods_shp['NAME'].unique()):
    neighborhood_id = neighborhoods_shp.loc[neighborhood_index]['OBJECTID']
    poly_shapes, pts, poly_to_pt_assignments = voronoi_within_neighborhood(neighborhoods_shp, neighborhood_name, n_points=15)

    for index, pp in enumerate(poly_shapes):
        new_row = {'CentroidID':int(index+1), 'centroid_lng':pp.centroid.x, 'centroid_lat':pp.centroid.y, 'NeighborhoodID':neighborhood_id, 'neighborhood_name':neighborhood_name, 'geometry':pp}
        partitioned_neighborhood_centroids = partitioned_neighborhood_centroids.append(new_row, ignore_index=True)
    
partitioned_neighborhood_centroids.head()

,CentroidID,NeighborhoodID,centroid_lat,centroid_lng,geometry,neighborhood_name
0,1.0,1.0,37.581518,-122.054469,"POLYGON ((-122.05677 37.58162, -122.05531 37.5...",Lakes and Birds
1,2.0,1.0,37.583721,-122.056755,"POLYGON ((-122.05531 37.58281, -122.05677 37.5...",Lakes and Birds
2,3.0,1.0,37.581641,-122.059747,"POLYGON ((-122.05677 37.58162, -122.05674 37.5...",Lakes and Birds
3,4.0,1.0,37.577701,-122.059779,"POLYGON ((-122.06083 37.57974, -122.05939 37.5...",Lakes and Birds
4,5.0,1.0,37.585313,-122.060500,"POLYGON ((-122.05908 37.58342, -122.06228 37.5...",Lakes and Birds


In [6]:
print('We created {x} centroids within {y} neighborhoods.'.format(x=repr(len(partitioned_neighborhood_centroids)), y=repr(len(neighborhoods_shp['NAME'].unique()))))

We created 218 centroids within 28 neighborhoods.


## 3. Visual check of generated centroid zones within Fremont Neighborhoods

In [7]:
from keplergl import KeplerGl
fremont_map = KeplerGl(height=600)
fremont_map.add_data(data=partitioned_neighborhood_centroids, name="Neighborhoods split into multiple centroid zones")
fremont_map.add_data(data=neighborhoods_shp, name="Fremont neighborhoods")
fremont_map

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(data={'Neighborhoods split into multiple centroid zones': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10…

***

## 4. Export the neighborhoods centroid zones

In [10]:
partitioned_neighborhood_centroids.to_csv(aux_files+'/Demand/OD demand/neighborhoods_centroid_zones.csv')